In [2]:
!pip install pandas sqlalchemy mysql-connector-python

# Agregated Transcation Data for SQL 

In [4]:
import os
import json
import mysql.connector

# ✅ Path to transaction data folder
base_path = r"C:/Users/LENOVO/Desktop/Phone pe/pulse-master/pulse-master/data/aggregated/transaction/country/india/state"

# ✅ MySQL Connection
connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Root',  # 👈 change if needed
    database='phonepe'
)
cursor = connection.cursor()

# ✅ Loop through folders: state/year/quarter.json
for state in os.listdir(base_path):
    state_path = os.path.join(base_path, state)
    for year in os.listdir(state_path):
        year_path = os.path.join(state_path, year)
        for quarter_file in os.listdir(year_path):
            file_name = quarter_file.strip('.json')
            if not file_name.isdigit():
                continue  # skip any unwanted files

            quarter = int(file_name)
            file_path = os.path.join(year_path, quarter_file)

            with open(file_path, 'r') as f:
                data = json.load(f)

            if 'data' in data and 'transactionData' in data['data']:
                for item in data['data']['transactionData']:
                    txn_type = item['name']
                    txn_count = item['paymentInstruments'][0]['count']
                    txn_amount = item['paymentInstruments'][0]['amount']

                    insert_query = """
                    INSERT INTO aggregated_transaction (state, year, quarter, transaction_type, transaction_count, transaction_amount)
                    VALUES (%s, %s, %s, %s, %s, %s)
                    """
                    values = (state, int(year), quarter, txn_type, txn_count, txn_amount)
                    cursor.execute(insert_query, values)

# ✅ Finalize
connection.commit()
cursor.close()
connection.close()

print("✅ Transaction data loaded successfully!")


✅ Transaction data loaded successfully!


# Agregated_User Data for SQL 

In [6]:
import os
import json
import mysql.connector

# ✅ Correct path
base_path = r"C:/Users/LENOVO/Desktop/Phone pe/pulse-master/pulse-master/data/aggregated/user/country/india/state"

# ✅ MySQL connection
connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Root',
    database='phonepe'
)
cursor = connection.cursor()

for state in os.listdir(base_path):
    state_path = os.path.join(base_path, state)
    for year in os.listdir(state_path):
        year_path = os.path.join(state_path, year)
        for quarter_file in os.listdir(year_path):
            file_name = quarter_file.strip('.json')
            if not file_name.isdigit():
                continue

            quarter = int(file_name)
            file_path = os.path.join(year_path, quarter_file)

            with open(file_path, 'r') as f:
                data = json.load(f)

            if 'data' in data and 'aggregated' in data['data']:
                registered_users = data['data']['aggregated'].get('registeredUsers', 0)
                app_opens = data['data']['aggregated'].get('appOpens', 0)

                insert_query = """
                INSERT INTO aggregated_user (state, year, quarter, registered_users, app_opens)
                VALUES (%s, %s, %s, %s, %s)
                """
                values = (state, int(year), quarter, registered_users, app_opens)
                cursor.execute(insert_query, values)

connection.commit()
cursor.close()
connection.close()

print("✅ Aggregated user data loaded successfully!")


✅ Aggregated user data loaded successfully!


# aggregated_insurance Data for SQL 

In [8]:
import os
import json
import mysql.connector

# ✅ Correct path
base_path = r"C:/Users/LENOVO/Desktop/Phone pe/pulse-master/pulse-master/data/aggregated/insurance/country/india/state"

# ✅ MySQL connection
connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Root',
    database='phonepe'
)
cursor = connection.cursor()

for state in os.listdir(base_path):
    state_path = os.path.join(base_path, state)
    for year in os.listdir(state_path):
        year_path = os.path.join(state_path, year)
        for quarter_file in os.listdir(year_path):
            file_name = quarter_file.strip('.json')
            if not file_name.isdigit():
                continue

            quarter = int(file_name)
            file_path = os.path.join(year_path, quarter_file)

            with open(file_path, 'r') as f:
                data = json.load(f)

            if 'data' in data and 'transactionData' in data['data']:
                for item in data['data']['transactionData']:
                    insurance_type = item['name']
                    insurance_count = item['paymentInstruments'][0]['count']
                    insurance_amount = item['paymentInstruments'][0]['amount']

                    insert_query = """
                    INSERT INTO aggregated_insurance (state, year, quarter, insurance_type, insurance_count, insurance_amount)
                    VALUES (%s, %s, %s, %s, %s, %s)
                    """
                    values = (state, int(year), quarter, insurance_type, insurance_count, insurance_amount)
                    cursor.execute(insert_query, values)

connection.commit()
cursor.close()
connection.close()

print("✅ Aggregated insurance data loaded successfully!")


✅ Aggregated insurance data loaded successfully!


# Map_transaction Data for SQL 

In [14]:
# "C:/Users/LENOVO/Desktop/Phone pe/pulse-master/pulse-master/data/map/transaction/hover/country/india/state"

import os
import json
import mysql.connector

base_path = r"C:/Users/LENOVO/Desktop/Phone pe/pulse-master/pulse-master/data/map/transaction/hover/country/india/state"

conn = mysql.connector.connect(host='localhost', user='root', password='Root', database='phonepe')
cur = conn.cursor()

for state in os.listdir(base_path):
    state_path = os.path.join(base_path, state)
    for year in os.listdir(state_path):
        year_path = os.path.join(state_path, year)
        for quarter_file in os.listdir(year_path):
            if not quarter_file.endswith('.json'):
                continue

            quarter = int(quarter_file.strip('.json'))
            file_path = os.path.join(year_path, quarter_file)

            # print(f"📖 Reading file: {file_path}")

            with open(file_path, 'r') as f:
                data = json.load(f)

            hover_data = data.get('data', {}).get('hoverDataList') or []

            if not hover_data:
                # print(f"⚠️ No hoverDataList in: {file_path}")
                continue

            for item in hover_data:
                district = item.get('name')
                txn_count = item.get('metric', [{}])[0].get('count', 0)
                txn_amount = item.get('metric', [{}])[0].get('amount', 0.0)

                # print(f"📝 Inserting: {state} | {year} | Q{quarter} | {district} | {txn_count} | {txn_amount}")

                cur.execute("""
                    INSERT INTO map_transaction (state, year, quarter, district, transaction_count, transaction_amount)
                    VALUES (%s, %s, %s, %s, %s, %s)
                """, (state, int(year), quarter, district, txn_count, txn_amount))

conn.commit()
cur.close()
conn.close()

print("✅ map_transaction loaded")



✅ map_transaction loaded


# Map_User Data for SQL 

In [16]:
import os
import json
import mysql.connector

base_path = r"C:/Users/LENOVO/Desktop/Phone pe/pulse-master/pulse-master/data/map/user/hover/country/india/state"

conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Root',
    database='phonepe'
)
cur = conn.cursor()

for state in os.listdir(base_path):
    state_path = os.path.join(base_path, state)
    for year in os.listdir(state_path):
        year_path = os.path.join(state_path, year)
        for file in os.listdir(year_path):
            if not file.endswith(".json"):
                continue

            quarter = int(file.strip(".json"))
            file_path = os.path.join(year_path, file)

            # print(f"\n📂 Reading: {file_path}")

            with open(file_path, 'r') as f:
                data = json.load(f)

            hover_data = data.get("data", {}).get("hoverData", {})
            # print(f"🟡 hoverData found with {len(hover_data)} districts")

            for district, values in hover_data.items():
                registered_users = values.get("registeredUsers", 0)

                # print(f"📌 Inserting: {state}, {year}, Q{quarter}, {district}, {registered_users}")

                cur.execute("""
                    INSERT INTO map_user (state, year, quarter, district, registered_users)
                    VALUES (%s, %s, %s, %s, %s)
                """, (state, int(year), quarter, district, registered_users))

conn.commit()
cur.close()
conn.close()

print("\n✅ map_user data load complete")



✅ map_user data load complete


# Map_insurance Data for SQL 

In [24]:
import os
import json
import mysql.connector

# ✅ CORRECTED PATH
base_path = r"C:/Users/LENOVO/Desktop/Phone pe/pulse-master/pulse-master/data/map/insurance/hover/country/india/state"

conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Root',
    database='phonepe'
)
cur = conn.cursor()

for state in os.listdir(base_path):
    state_path = os.path.join(base_path, state)
    for year in os.listdir(state_path):
        year_path = os.path.join(state_path, year)
        for file in os.listdir(year_path):
            if not file.endswith(".json"):
                continue

            quarter = int(file.strip(".json"))
            file_path = os.path.join(year_path, file)

            with open(file_path, 'r') as f:
                data = json.load(f)

            hover_data_list = data.get("data", {}).get("hoverDataList", [])

            if not hover_data_list:
                # print(f"⚠️ Empty hoverDataList in: {file_path}")
                continue

            for item in hover_data_list:
                district = item.get("name")
                metric = item.get("metric", [{}])[0]
                insured_users = metric.get("count", 0)

                # print(f"🟢 Inserting: {state} | {year} | Q{quarter} | {district} | insured_users: {insured_users}")

                cur.execute("""
                    INSERT INTO map_insurance (state, year, quarter, district, insured_users)
                    VALUES (%s, %s, %s, %s, %s)
                """, (state, int(year), quarter, district, insured_users))

conn.commit()
cur.close()
conn.close()

print("\n✅ map_insurance data loaded successfully!")



✅ map_insurance data loaded successfully!


# Top_transaction Data for SQL 

In [26]:
import os
import json
import mysql.connector

# ✅ Path to top transaction JSONs
base_path = r"C:/Users/LENOVO/Desktop/Phone pe/pulse-master/pulse-master/data/top/transaction/country/india/state"

# ✅ Connect to MySQL
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Root',
    database='phonepe'
)
cur = conn.cursor()

# ✅ Loop through state → year → quarter
for state in os.listdir(base_path):
    state_path = os.path.join(base_path, state)
    for year in os.listdir(state_path):
        year_path = os.path.join(state_path, year)
        for file in os.listdir(year_path):
            if not file.endswith(".json"):
                continue

            quarter = int(file.strip(".json"))
            file_path = os.path.join(year_path, file)

            # print(f"\n📂 Reading: {file_path}")

            with open(file_path, 'r') as f:
                data = json.load(f)

            pin_data = data.get("data", {}).get("pincodes", [])
            # print(f"📦 Pincodes Found: {len(pin_data)}")

            for entry in pin_data:
                pincode = entry.get("entityName")
                txn_count = entry.get("metric", {}).get("count", 0)
                txn_amount = entry.get("metric", {}).get("amount", 0.0)

                # print(f"📌 Inserting: {state}, {year}, Q{quarter}, {pincode}, {txn_count}, {txn_amount}")

                cur.execute("""
                    INSERT INTO top_transaction (state, year, quarter, pincode, transaction_count, transaction_amount)
                    VALUES (%s, %s, %s, %s, %s, %s)
                """, (state, int(year), quarter, pincode, txn_count, txn_amount))

# ✅ Done
conn.commit()
cur.close()
conn.close()

print("\n✅ top_transaction data loaded successfully!")



✅ top_transaction data loaded successfully!


# Top_user Data for SQL 

In [28]:
import os
import json
import mysql.connector

# ✅ Path to your JSON data
base_path = r"C:/Users/LENOVO/Desktop/Phone pe/pulse-master/pulse-master/data/top/user/country/india/state"

# ✅ Connect to MySQL
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Root',
    database='phonepe'
)
cur = conn.cursor()

# ✅ Loop through states, years, and quarters
for state in os.listdir(base_path):
    state_path = os.path.join(base_path, state)
    for year in os.listdir(state_path):
        year_path = os.path.join(state_path, year)
        for file in os.listdir(year_path):
            if not file.endswith(".json"):
                continue

            quarter = int(file.strip(".json"))
            file_path = os.path.join(year_path, file)

            # print(f"\n📂 Reading: {file_path}")

            with open(file_path, 'r') as f:
                data = json.load(f)

            pin_data = data.get("data", {}).get("pincodes", [])
            # print(f"📦 Pincodes Found: {len(pin_data)}")

            for entry in pin_data:
                pincode = entry.get("name")
                registered_users = entry.get("registeredUsers", 0)

                # print(f"📌 Inserting: {state}, {year}, Q{quarter}, {pincode}, {registered_users}")

                cur.execute("""
                    INSERT INTO top_user (state, year, quarter, pincode, registered_users)
                    VALUES (%s, %s, %s, %s, %s)
                """, (state, int(year), quarter, pincode, registered_users))

# ✅ Save and close
conn.commit()
cur.close()
conn.close()

print("\n✅ top_user data loaded successfully!")



✅ top_user data loaded successfully!


# Top_insurance_new Data for SQL 


In [32]:
import os
import json
import mysql.connector

# ✅ Correct path to top/insurance data
base_path = r"C:/Users/LENOVO/Desktop/Phone pe/pulse-master/pulse-master/data/top/insurance/country/india/state"

# ✅ MySQL connection
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Root',
    database='phonepe'
)
cur = conn.cursor()

# ✅ Loop through state → year → quarter
for state in os.listdir(base_path):
    state_path = os.path.join(base_path, state)
    for year in os.listdir(state_path):
        year_path = os.path.join(state_path, year)
        for file in os.listdir(year_path):
            if not file.endswith(".json"):
                continue

            quarter = int(file.strip(".json"))
            file_path = os.path.join(year_path, file)

            with open(file_path, 'r') as f:
                data = json.load(f)

            pin_data = data.get("data", {}).get("pincodes", [])

            for entry in pin_data:
                pincode = entry.get("name")  # NOTE: sometimes "name" is used
                insured_users = entry.get("registeredUsers", 0)

                # ✅ Insert into DB
                cur.execute("""
                    INSERT INTO top_insurance (state, year, quarter, pincode, insured_users)
                    VALUES (%s, %s, %s, %s, %s)
                """, (state, int(year), quarter, pincode, insured_users))

# ✅ Finalize
conn.commit()
cur.close()
conn.close()

print("✅ top_insurance_new data loaded successfully!")


✅ top_insurance_new data loaded successfully!


# Final Check data import or not 


In [34]:
import mysql.connector

# ✅ MySQL Connection
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Root',
    database='phonepe'
)
cur = conn.cursor()

tables = [
    "aggregated_transaction",
    "aggregated_user",
    "aggregated_insurance",
    "map_transaction",
    "map_user",
    "map_insurance",
    "top_transaction",
    "top_user",
    "top_insurance"
]

print("📊 Final Table Row Count Check:\n")

for table in tables:
    try:
        cur.execute(f"SELECT COUNT(*) FROM {table}")
        count = cur.fetchone()[0]
        print(f"✅ {table}: {count} rows")
    except Exception as e:
        print(f"❌ {table}: Error - {e}")

cur.close()
conn.close()


📊 Final Table Row Count Check:

✅ aggregated_transaction: 5034 rows
✅ aggregated_user: 1008 rows
✅ aggregated_insurance: 682 rows
✅ map_transaction: 20604 rows
✅ map_user: 20608 rows
✅ map_insurance: 13876 rows
✅ top_transaction: 9999 rows
✅ top_user: 10000 rows
✅ top_insurance: 6668 rows
